# Spark Initialization

In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME
import findspark
findspark.init()

In [2]:
# Import required library
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
# Print Spark object ID
print(spark)

# Loading Data using Spark

In [4]:
# Data dapat di download di https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales
# Ukuran file 136MB

df = spark.read.csv("F:/Semester 6/BigData/DataSet/1-6m-accidents-traffic-flow-over-16-years/accidents_2012_to_2014.csv", header=True, inferSchema=True)

In [5]:
# Jumlah data

df.count()

464697

In [6]:
# Menampilkan data

df.show()

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-----+--------------------------+-------------------------+--------------+---------------+------------------+-----------+---------------+--------------------+--------------+---------------+---------------------------------+---------------------------------------+--------------------+--------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+----+
|Accident_Index|Location_Easting_OSGR|Location_Northing_OSGR|Longitude| Latitude|Police_Force|Accident_Severity|Number_of_Vehicles|Number_of_Casualties|      Date|Day_of_Week| Time|Local_Authority_(District)|Local_Authority_(Highway)|1st_Road_Class|1st_Road_Number|         Road_Type|Speed_limit|Junction_Detail|    Junction_Control|2nd_Road_Class|2nd_R

In [7]:
# Data Types

df.schema

StructType(List(StructField(Accident_Index,StringType,true),StructField(Location_Easting_OSGR,IntegerType,true),StructField(Location_Northing_OSGR,IntegerType,true),StructField(Longitude,DoubleType,true),StructField(Latitude,DoubleType,true),StructField(Police_Force,IntegerType,true),StructField(Accident_Severity,IntegerType,true),StructField(Number_of_Vehicles,IntegerType,true),StructField(Number_of_Casualties,IntegerType,true),StructField(Date,StringType,true),StructField(Day_of_Week,IntegerType,true),StructField(Time,StringType,true),StructField(Local_Authority_(District),IntegerType,true),StructField(Local_Authority_(Highway),StringType,true),StructField(1st_Road_Class,IntegerType,true),StructField(1st_Road_Number,IntegerType,true),StructField(Road_Type,StringType,true),StructField(Speed_limit,IntegerType,true),StructField(Junction_Detail,StringType,true),StructField(Junction_Control,StringType,true),StructField(2nd_Road_Class,IntegerType,true),StructField(2nd_Road_Number,IntegerTy

In [8]:
# Register the DataFrame as a SQL temporary view

df.createOrReplaceTempView("accidents")

In [9]:
result = spark.sql("SELECT DISTINCT Road_Surface_Conditions FROM accidents")

In [10]:
result.show()

+-----------------------+
|Road_Surface_Conditions|
+-----------------------+
|   Flood (Over 3cm o...|
|              Frost/Ice|
|                   null|
|               Wet/Damp|
|                    Dry|
|                   Snow|
+-----------------------+



# Data Mining Process

In [11]:
# 1. Jumlah kecelakaan dalam tiap hari

query1 = spark.sql("SELECT TO_DATE(`Date`, 'MM/DD/YYYY') AS date, COUNT(`Date`) \
                    FROM accidents \
                    GROUP BY `Date` \
                    ORDER BY `Date` DESC")

In [12]:
query1.show()

+----------+-----------+
|      date|count(Date)|
+----------+-----------+
|2012-01-01|        510|
|2012-01-01|        550|
|2012-01-01|        572|
|2012-01-01|        669|
|2012-01-01|        317|
|2012-01-01|        489|
|2012-01-01|        540|
|2012-01-01|        424|
|2012-01-01|        631|
|2012-01-01|        678|
|2012-01-01|        516|
|2012-01-01|        392|
|2012-01-01|        548|
|2012-01-01|        356|
|2012-01-01|        322|
|2012-01-01|        504|
|2012-01-01|        536|
|2012-01-01|        457|
|2012-01-01|        485|
|2012-01-01|        631|
+----------+-----------+
only showing top 20 rows



In [13]:
# 2. Jumlah kecelakaan yang didatangi polisi berdasarkan keparahan kecelakaan
# 1 Fatal
# 2 Serious
# 3 Slight

query2 = spark.sql("SELECT Accident_Severity, COUNT(Did_Police_Officer_Attend_Scene_of_Accident) as Jumlah_Yes \
                    FROM accidents \
                    WHERE Did_Police_Officer_Attend_Scene_of_Accident='Yes'\
                    GROUP BY Accident_Severity \
                    ORDER BY COUNT(Accident_Severity) DESC")

In [14]:
query2.show()

+-----------------+----------+
|Accident_Severity|Jumlah_Yes|
+-----------------+----------+
|                3|    315162|
|                2|     60449|
|                1|      5239|
+-----------------+----------+



In [15]:
# 3. Pengaruh cuaca terhadap keparahan kecelakaan

query3 = spark.sql("SELECT Accident_Severity, Weather_Conditions, COUNT(Accident_Severity) as Jumlah \
                    FROM accidents \
                    GROUP BY Accident_Severity, Weather_Conditions \
                    ORDER BY Accident_Severity ASC")

In [16]:
query3.show()

+-----------------+--------------------+------+
|Accident_Severity|  Weather_Conditions|Jumlah|
+-----------------+--------------------+------+
|                1|Snowing without h...|    24|
|                1|             Unknown|    54|
|                1|         Fog or mist|    53|
|                1|Fine with high winds|    79|
|                1|Raining without h...|   512|
|                1|Snowing with high...|     3|
|                1|               Other|    61|
|                1|Raining with high...|    81|
|                1|Fine without high...|  4436|
|                2|Snowing without h...|   285|
|                2|Snowing with high...|    78|
|                2|               Other|  1068|
|                2|Fine without high...| 55006|
|                2|Fine with high winds|   756|
|                2|             Unknown|   964|
|                2|Raining with high...|   949|
|                2|         Fog or mist|   355|
|                2|Raining without h...|

In [17]:
# 4. Pengaruh tipe jalan terhadap speed limit

query4 = spark.sql("SELECT DISTINCT Road_Type, Speed_limit \
                    FROM accidents \
                    GROUP BY Road_Type, Speed_limit")

In [18]:
query4.show()

+------------------+-----------+
|         Road_Type|Speed_limit|
+------------------+-----------+
|  Dual carriageway|         50|
|        Roundabout|         60|
|    One way street|         40|
|Single carriageway|         10|
|    One way street|         60|
|         Slip road|         60|
|        Roundabout|         30|
|Single carriageway|         20|
|         Slip road|         40|
|  Dual carriageway|         40|
|           Unknown|         70|
|Single carriageway|         60|
|         Slip road|         50|
|           Unknown|         30|
|Single carriageway|         50|
|        Roundabout|         70|
|        Roundabout|         20|
|        Roundabout|         40|
|           Unknown|         60|
|         Slip road|         70|
+------------------+-----------+
only showing top 20 rows



In [19]:
# 5. Pengaruh fasilitas terdekat yang ada dengan jumlah korban

query5 = spark.sql("SELECT `Pedestrian_Crossing-Physical_Facilities`, Number_of_Casualties, COUNT(Number_of_Casualties) as Jumlah \
                    FROM accidents \
                    GROUP BY `Pedestrian_Crossing-Physical_Facilities`, Number_of_Casualties \
                    ORDER BY Number_of_Casualties ASC")

In [20]:
query5.show()

+---------------------------------------+--------------------+------+
|Pedestrian_Crossing-Physical_Facilities|Number_of_Casualties|Jumlah|
+---------------------------------------+--------------------+------+
|                   No physical cross...|                   1|290019|
|                         Central refuge|                   1|  8381|
|                         Zebra crossing|                   1| 12111|
|                   non-junction pede...|                   1| 22072|
|                   Pedestrian phase ...|                   1| 27885|
|                   Footbridge or subway|                   1|  1158|
|                         Zebra crossing|                   2|  1418|
|                   Footbridge or subway|                   2|   231|
|                         Central refuge|                   2|  1430|
|                   No physical cross...|                   2| 59571|
|                   non-junction pede...|                   2|  3203|
|                   

In [23]:
# 6. Jumlah kondisi spesial yang terjadi

query6 = spark.sql("SELECT Special_Conditions_at_Site, COUNT(Special_Conditions_at_Site) as Jumlah \
                    FROM accidents \
                    GROUP BY Special_Conditions_at_Site \
                    ORDER BY Jumlah DESC")

In [24]:
query6.show()

+--------------------------+------+
|Special_Conditions_at_Site|Jumlah|
+--------------------------+------+
|                      None|454385|
|                 Roadworks|  4742|
|                       Mud|  1413|
|              Ol or diesel|  1406|
|      Road surface defe...|  1144|
|      Auto traffic sing...|   758|
|      Permanent sign or...|   625|
|      Auto traffic sign...|   222|
|                      null|     0|
+--------------------------+------+

